In [71]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import datetime


path = Path.cwd().parent
path = str(path)+'/GFMS DB/'
path

'/Users/keumhyeonjun/Projects/행안부/자재/GFMS DB/'

# 데이터 불러오기

In [114]:
# 불러오는 데이터의 날짜(작업장비이력 테이블의 가장 최근 날짜)


FMSA10_작업지시_결과 = pd.read_csv(path+'FMSA10_작업지시_결과_ver2.csv')

FMSA11_작업장비이력 = pd.read_excel(path + 'FMSA11_작업장비이력.xlsx')

FMSX05_장비마스터 = pd.read_csv(path + 'FMSX05_장비마스터_ver2.csv')

FMSB07_자재출고 = pd.read_excel(path + 'FMSB07_자재출고.xlsx')

FMSB03_자재마스터 = pd.read_csv(path + 'FMSB03_자재마스터_ver2.csv')

FMSB03_자재마스터.drop(['class_nm_4','class_cd_4'],axis =1,inplace = True)

FMSB03_자재마스터.rename(columns={'class_nm_3':'jajae_class_nm_3'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_nm_2':'jajae_class_nm_2'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_nm_1':'jajae_class_nm_1'},inplace = True)

FMSB03_자재마스터.rename(columns={'class_cd_3':'jajae_class_cd_3'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_cd_2':'jajae_class_cd_2'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_cd_1':'jajae_class_cd_1'},inplace = True)


FMSX05_장비마스터.rename(columns={'class_nm_4':'jangbi_class_nm_4'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_3':'jangbi_class_nm_3'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_2':'jangbi_class_nm_2'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_1':'jangbi_class_nm_1'},inplace = True)
FMSX01_건물정보 = pd.read_excel(path+'FMSX01_건물정보.xlsx')
fmsx01 = FMSX01_건물정보[['bd_cd','bd_nm']]
FMSX05_장비마스터 = pd.merge(fmsx01,FMSX05_장비마스터, on='bd_cd', how = 'right')


FMSX05_장비마스터.rename(columns={'class_cd_4':'jangbi_class_cd_4'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_3':'jangbi_class_cd_3'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_2':'jangbi_class_cd_2'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_1':'jangbi_class_cd_1'},inplace = True)

FMSA10_작업지시_결과.rename(columns={'class_nm_4':'job_class_nm_4'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_3':'job_class_nm_3'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_2':'job_class_nm_2'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_1':'job_class_nm_1'},inplace = True)


FMSA10_작업지시_결과.rename(columns={'class_cd_4':'job_class_cd_4'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_3':'job_class_cd_3'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_2':'job_class_cd_2'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_1':'job_class_cd_1'},inplace = True)

fmsa10 =FMSA10_작업지시_결과[['job_cd','job_nm','job_jisi_dt','job_enddt','job_shm','job_ehm','job_class_nm_1','job_class_nm_2','job_class_nm_3','job_class_nm_4']]
fmsb03 = FMSB03_자재마스터[['mat_cd','mat_nm','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]
fmsx05 = FMSX05_장비마스터[['bd_cd','bd_nm','fl_cd','fac_cd','fac_nm','jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4','fac_instdt']]


FMSB07_자재출고["mat_cd"] = FMSB07_자재출고["mat_cd"].astype("str")
FMSB07_자재출고["mat_cd"] = list(map(lambda x: x[:-3],FMSB07_자재출고["mat_cd"].tolist())) # FMSB03과 inner join하기 위해 mat_cd 형식 일치
fmsb03["mat_cd"] = fmsb03["mat_cd"].astype("str")
jajae = pd.merge(FMSB07_자재출고,fmsb03 ,on = 'mat_cd', how = 'inner')
jajae = jajae[['job_cd','outw_day','mat_cd','outw_qty','mat_nm_x','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]


jangbi = pd.merge(FMSA11_작업장비이력, fmsx05, how='inner', on='fac_cd')
jangbi_broken = pd.merge(fmsa10,jangbi, on='job_cd',how = 'inner')

jangbi_broken.drop(['job_cont','chg_sysdt','chg_emp_no'],axis=1,inplace=True)
jangbi_broken = jangbi_broken.dropna()

jangbi_broken['자재매칭여부'] = jangbi_broken['job_cd']
arr_장비 = list(jangbi_broken['job_cd'])
arr_자재 = list(jajae['job_cd'])
#자재 매칭이 안되는 경우
arr = list(set(arr_장비)- set(arr_자재))

def function(x):
    if x in arr:
        return 0
    else:
         return 1

jangbi_broken['자재매칭여부'] = jangbi_broken['자재매칭여부'].apply(function)
df = pd.merge(jangbi_broken,jajae, on='job_cd',how = 'inner')

final = df[['job_cd','job_nm','job_jisi_dt','job_enddt','job_shm','job_ehm','job_class_nm_1','job_class_nm_2','job_class_nm_3','job_class_nm_4','fac_cd','fac_nm','fac_instdt','mat_cd','mat_nm_x',
           'jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4','자재매칭여부','mat_nm_x','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]


jangbi_broken = jangbi_broken.sort_values('job_jisi_dt')

## 기준일자는 오늘로 설정해야 함 > 작업장비이력
jangbi_broken['기준일자'] = '2022-10-11'

jangbi_broken['기준일자'] = pd.to_datetime(jangbi_broken['기준일자'])
jangbi_broken['fac_instdt'] = pd.to_datetime(jangbi_broken['fac_instdt'])
jangbi_broken['job_jisi_dt'] = pd.to_datetime(jangbi_broken['job_jisi_dt'])


jangbi_broken['설치-작업간격'] = jangbi_broken['job_jisi_dt'] - jangbi_broken['fac_instdt']

def slicing(x):
    return x[:-4]
jangbi_broken['설치-작업간격'] = jangbi_broken['설치-작업간격'].astype(str)
jangbi_broken['설치-작업간격'] = jangbi_broken['설치-작업간격'].apply(slicing)
jangbi_broken['설치-작업간격'] = jangbi_broken['설치-작업간격'].astype(int)

jangbi_broken['작업-작업간격'] = 0
# 모두 장애가 발생한 장비이므로 작업횟수 = 1 로 세팅
jangbi_broken['작업횟수'] = 1
jangbi_broken = jangbi_broken.sort_values(['fac_cd','설치-작업간격'])

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_42442/324639401.py:4: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  FMSA10_작업지시_결과 = pd.read_csv(path+'FMSA10_작업지시_결과_ver2.csv')
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_42442/324639401.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fmsb03["mat_cd"] = fmsb03["mat_cd"].astype("str")


In [115]:

# 작업-작업 간격 만들기
for i in range(len(jangbi_broken)):
    if i == 0:
        # 젤 처음 오는 경우 > 장비별 가장 오래된 작업
        jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i]
    else :
        now_data = jangbi_broken['fac_cd'].iloc[i]
        past_data = jangbi_broken['fac_cd'].iloc[i-1]
        # 한 장비가 이전에 장애가 발생했을 경우
        if now_data == past_data:
            jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i] - jangbi_broken['설치-작업간격'].iloc[i-1]
            jangbi_broken['작업횟수'].iloc[i] = jangbi_broken['작업횟수'].iloc[i-1] +1
        # 장비가 이전에 장애가 발생하지 않았을 경우
        else :
            jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i]

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_42442/279619396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i]
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_42442/279619396.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i] - jangbi_broken['설치-작업간격'].iloc[i-1]
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_42442/279619396.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

In [116]:
max_time = datetime.datetime(2022,10,11)

analysis_data = pd.merge(fmsx05,jangbi_broken[['job_jisi_dt','fac_cd','설치-작업간격','작업-작업간격','작업횟수','job_cd','job_nm']], on ='fac_cd', how ='left')

analysis_data['fac_instdt'] = pd.to_datetime(analysis_data['fac_instdt'])

analysis_data = analysis_data[analysis_data['fac_instdt'] <= max_time]


analysis_data['max_time'] = max_time
analysis_data['max_time'] = pd.to_datetime(analysis_data['max_time'])
analysis_data['작업-작업간격'] = analysis_data['작업-작업간격'].fillna(999999)

analysis_data['설치-작업간격'] = analysis_data['설치-작업간격'].fillna(999999)

analysis_data['경과시간'] = 0

def time(df):
    x = df['작업-작업간격']
    if x == 999999:
        return int(str(df['max_time'] -  df['fac_instdt'])[:-13])
    else :
        return int(x)
def time2(df):
    x = df['설치-작업간격']
    if x == 999999:
        return int(str(df['max_time'] -  df['fac_instdt'])[:-13])
    else :
        return int(x)

def time3(df):
    if df['status'] == 0 :
        return int(str(df['max_time'] -  df['fac_instdt'])[:-13])
    else :
        return int(str(df['max_time'] -  df['job_jisi_dt'])[:-13])


def status(df):
    if df['작업횟수'] == 0:
        return 0
    else :
        return 1

analysis_data['작업횟수'] = analysis_data['작업횟수'].fillna(0)
analysis_data['작업-작업간격'] = analysis_data.apply(time,axis=1)
analysis_data['설치-작업간격'] = analysis_data.apply(time2,axis=1)
analysis_data['status'] = analysis_data.apply(status,axis=1)
analysis_data['경과시간'] =  analysis_data.apply(time3,axis=1)
analysis_data = analysis_data.fillna(0)
analysis_data

,bd_cd,bd_nm,fl_cd,fac_cd,fac_nm,jangbi_class_nm_1,jangbi_class_nm_2,jangbi_class_nm_3,jangbi_class_nm_4,fac_instdt,job_jisi_dt,설치-작업간격,작업-작업간격,작업횟수,job_cd,job_nm,max_time,경과시간,status
0,0,0,0,200201040293,(3)세-1F-0(여)로비-화장실) 세면대 2대,기계,"위생, 탱크설비",위생설비,세면대 수전,2018-03-05,0,1681,1681,0.0,0,0,2022-10-11,1681,0
1,0,0,0,200201010257,9F여1대변기(5),기계,"위생, 탱크설비",위생설비,대변기,2014-06-01,0,3054,3054,0.0,0,0,2022-10-11,3054,0
2,0,0,0,200202030001,우수조#1,기계,"위생, 탱크설비",탱크설비,우수 저장조,2019-10-22,0,1085,1085,0.0,0,0,2022-10-11,1085,0
3,0,0,0,200105080599,배기휀#76(EF-504),기계,공기조화설비,환기설비,환기팬,2019-10-23,0,1084,1084,0.0,0,0,2022-10-11,1084,0
4,0,0,0,200202010001,시수조#1,기계,"위생, 탱크설비",탱크설비,상수 저장조,2019-10-22,0,1085,1085,0.0,0,0,2022-10-11,1085,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43584,SJ-SCJS-01,SCC전시동,P006,700101020004,SCC 4호기 권상기,승강기,승객용,MR,권상기,2013-11-12,0,3255,3255,0.0,0,0,2022-10-11,3255,0
43585,SJ-SCJS-01,SCC전시동,P006,700101020005,SCC 5호기 권상기,승강기,승객용,MR,권상기,2013-11-12,0,3255,3255,0.0,0,0,2022-10-11,3255,0
43586,SJ-SCJS-01,SCC전시동,P006,700101020006,SCC 6호기 권상기,승강기,승객용,MR,권상기,2013-11-12,0,3255,3255,0.0,0,0,2022-10-11,3255,0
43587,SJ-SCJS-01,SCC전시동,P006,700201010010,SCC 7호기 제어반,승강기,화물용,로프식,제어반,2014-10-17,0,2916,2916,0.0,0,0,2022-10-11,2916,0


In [117]:
기계 = pd.read_excel('수행자재_완료.xlsx',sheet_name='기계')
통신 = pd.read_excel('수행자재_완료.xlsx',sheet_name='통신')
방재 = pd.read_excel('수행자재_완료.xlsx',sheet_name='방재')
승강기 = pd.read_excel('수행자재_완료.xlsx',sheet_name='승강기')
전기 = pd.read_excel('수행자재_완료.xlsx',sheet_name='전기')

In [118]:
수행범위 = pd.concat([기계,통신,방재,승강기,전기])
input_data= pd.merge(수행범위,analysis_data, how = 'left',left_on=['Lv1','Lv2','Lv3','Lv4'],right_on=['jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4'])
input_data.drop(['jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4'],axis=1,inplace = True)
input_data = input_data.dropna()

In [119]:
input_data

,Lv1,Lv2,Lv3,Lv4,장애건수,자재매칭건수,매칭비율,그룹정의,장애분석,자재분석,...,fac_instdt,job_jisi_dt,설치-작업간격,작업-작업간격,작업횟수,job_cd,job_nm,max_time,경과시간,status
0,기계,공기조화설비,개별냉난방설비,개별냉난방기 실내기,309.0,4.0,0.012945,개별냉난방기,O,X,...,2014-04-01,0,3115.0,3115.0,0.0,0,0,2022-10-11,3115.0,0.0
1,기계,공기조화설비,개별냉난방설비,개별냉난방기 실내기,309.0,4.0,0.012945,개별냉난방기,O,X,...,2019-11-01,0,1075.0,1075.0,0.0,0,0,2022-10-11,1075.0,0.0
2,기계,공기조화설비,개별냉난방설비,개별냉난방기 실내기,309.0,4.0,0.012945,개별냉난방기,O,X,...,2019-11-01,0,1075.0,1075.0,0.0,0,0,2022-10-11,1075.0,0.0
3,기계,공기조화설비,개별냉난방설비,개별냉난방기 실내기,309.0,4.0,0.012945,개별냉난방기,O,X,...,2013-09-01,0,3327.0,3327.0,0.0,0,0,2022-10-11,3327.0,0.0
4,기계,공기조화설비,개별냉난방설비,개별냉난방기 실내기,309.0,4.0,0.012945,개별냉난방기,O,X,...,2013-09-01,0,3327.0,3327.0,0.0,0,0,2022-10-11,3327.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38185,전기,전력부하설비,조명,옥외 LED 조명,27.0,12.0,0.440000,옥외 LED 조명,o,o,...,2013-11-01,0,3266.0,3266.0,0.0,0,0,2022-10-11,3266.0,0.0
38186,전기,전력부하설비,조명,옥외 LED 조명,27.0,12.0,0.440000,옥외 LED 조명,o,o,...,2013-11-01,0,3266.0,3266.0,0.0,0,0,2022-10-11,3266.0,0.0
38187,전기,전력부하설비,조명,옥외 LED 조명,27.0,12.0,0.440000,옥외 LED 조명,o,o,...,2013-11-01,0,3266.0,3266.0,0.0,0,0,2022-10-11,3266.0,0.0
38188,전기,전력부하설비,조명,옥외 LED 조명,27.0,12.0,0.440000,옥외 LED 조명,o,o,...,2013-11-01,0,3266.0,3266.0,0.0,0,0,2022-10-11,3266.0,0.0


In [120]:
input_data.to_csv('생존분석.csv')

- 장비분류(그룹핑), 동, 층, 장비명, 장애발생횟수, 최근장애일자 > 가져와서 돌리면돔

In [113]:
FMSX05_장비마스터

,bd_cd,bd_nm,Unnamed: 0,comp_cd,fac_cd,fac_nm,class_cd,dept_cd,fac_model,fac_spec,...,chk_edit_dt,law_fac_yn,jangbi_class_nm_1,jangbi_class_nm_2,jangbi_class_nm_3,jangbi_class_nm_4,jangbi_class_cd_1,jangbi_class_cd_2,jangbi_class_cd_3,jangbi_class_cd_4
0,NaN,NaN,0,SEJO,200201040293,(3)세-1F-0(여)로비-화장실) 세면대 2대,20020104,10201,자동(2)/수동(0),청소수전,...,NaN,N,기계,"위생, 탱크설비",위생설비,세면대 수전,20,2002,200201,20020104
1,NaN,NaN,1,SEJO,200201010257,9F여1대변기(5),20020101,10201,CC-113(바닥형),690*395*435,...,NaN,N,기계,"위생, 탱크설비",위생설비,대변기,20,2002,200201,20020101
2,NaN,NaN,2,SEJO,200202030001,우수조#1,20020203,10201,NaN,NaN,...,NaN,N,기계,"위생, 탱크설비",탱크설비,우수 저장조,20,2002,200202,20020203
3,NaN,NaN,3,SEJO,200105080599,배기휀#76(EF-504),20010508,10201,NaN,NaN,...,NaN,N,기계,공기조화설비,환기설비,환기팬,20,2001,200105,20010508
4,NaN,NaN,4,SEJO,200202010001,시수조#1,20020201,10201,NaN,NaN,...,NaN,N,기계,"위생, 탱크설비",탱크설비,상수 저장조,20,2002,200202,20020201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40806,SJ-SCJS-01,SCC전시동,40806,SEJO,700101020004,SCC 4호기 권상기,70010102,10302,HS250K,12*6,...,NaN,N,승강기,승객용,MR,권상기,70,7001,700101,70010102
40807,SJ-SCJS-01,SCC전시동,40807,SEJO,700101020005,SCC 5호기 권상기,70010102,10302,HS250K,12*6,...,NaN,N,승강기,승객용,MR,권상기,70,7001,700101,70010102
40808,SJ-SCJS-01,SCC전시동,40808,SEJO,700101020006,SCC 6호기 권상기,70010102,10302,HS250K,12*6,...,NaN,N,승강기,승객용,MR,권상기,70,7001,700101,70010102
40809,SJ-SCJS-01,SCC전시동,40809,SEJO,700201010010,SCC 7호기 제어반,70020101,10302,DZDLE-4300,NaN,...,NaN,N,승강기,화물용,로프식,제어반,70,7002,700201,70020101
